In [4]:
# Homework. Lesson 19

import pandas as pd

In [17]:
# 1) Analyzing Sales Data
df = pd.read_csv('sales_data.csv')

In [7]:
# Statistics for each category
category_stats = df.groupby('Category').agg(
    Total_quantity_sold = ('Quantity', 'sum'),
    Avarage_price_per_unit = ('Price', 'mean'),
    max_quantity_single_transaction = ('Quantity', 'max')
).reset_index()

print(category_stats)

      Category  Total_quantity_sold  Avarage_price_per_unit  \
0     Clothing                  157               31.176471   
1  Electronics                  183              276.764706   
2         Home                  144               55.000000   

   max_quantity_single_transaction  
0                               15  
1                               15  
2                               14  


In [9]:
# Top-selling product in each category
top_products = (df.groupby(['Category', 'Product'])['Quantity'].sum().reset_index()
                .sort_values(['Category', 'Quantity'], ascending=[True, False]))

top_products_each_category = top_products.groupby('Category').first().reset_index()
print(top_products_each_category)

      Category          Product  Quantity
0     Clothing            Jeans        15
1  Electronics         Smart TV        15
2         Home  Pressure Cooker        14


In [10]:
# Date on which the highest total sales (quantity * price) occurred
df['Sales'] = df['Quantity'] * df['Price']

daily_sales = df.groupby('Date')['Sales'].sum().reset_index()
max_sales_date = daily_sales.loc[daily_sales['Sales'].idxmax()]

print(max_sales_date)

Date     2023-01-07
Sales         15150
Name: 6, dtype: object


In [5]:
# 2) Examining Customer Orders
df = pd.read_csv('customer_orders.csv')


In [13]:
# Group the data by CustomerID and filter out customers who have made less than 20 orders
customer_order_counts = df.groupby('CustomerID')['OrderID'].nunique().reset_index(name='OrderCount')
customers_20plus = customer_order_counts[customer_order_counts['OrderCount'] >= 20]
print(customers_20plus)

   CustomerID  OrderCount
0         101          21
1         102          21
2         103          20
3         104          20


In [14]:
# Identify customers who have ordered products with an average price per unit greater than $120
customer_avg_price = df.groupby('CustomerID')['Price'].mean().reset_index(name="AvgPrice")
high_price_customers = customer_avg_price[customer_avg_price['AvgPrice'] > 120]
print(high_price_customers)

   CustomerID    AvgPrice
1         102  138.095238
3         104  169.750000


In [ ]:
# Find the total quantity and total price for each product ordered, and filter out products that have a total quantity less than 5 units
product_totals = df.groupby("Product").agg(
    TotalQuantity=("Quantity", "sum"),
    TotalPrice=("Price", lambda x: (x * df.loc[x.index, "Quantity"]).sum())
).reset_index()

filtered_products = product_totals[product_totals["TotalQuantity"] >= 5]
print(filtered_products)


=== 5 donadan ko'p sotilgan mahsulotlar ===
             Product  TotalQuantity  TotalPrice
5        Cargo Pants              6         180
15       Dress Shirt              5         125
19      Formal Shirt              6         210
30        Smartphone              5        2000
32       Sport Shoes              5         200
35        Sunglasses              5          75
41  Wireless Earbuds              6         720


In [18]:
# 3) Population Salary Analysis
import sqlite3
import numpy as np

# 1. SQLite dan o‘qish
conn = sqlite3.connect("population.db")
population_df = pd.read_sql("SELECT * FROM population", conn)
conn.close()

# 2. Salary bandlarni qo‘lbola belgilash
def categorize_salary(salary):
    if salary < 1000:
        return "Low"
    elif salary < 5000:
        return "Medium"
    else:
        return "High"

population_df["SalaryCategory"] = population_df["salary"].apply(categorize_salary)

# 3. Umumiy hisob-kitob
total_population = len(population_df)

salary_category_stats = (
    population_df.groupby("SalaryCategory")["salary"]
    .agg(
        PopulationCount="count",
        AverageSalary="mean",
        MedianSalary="median"
    )
    .reset_index()
)

salary_category_stats["PopulationPercentage"] = (
    salary_category_stats["PopulationCount"] / total_population * 100
)

print("=== Umumiy Salary Band bo'yicha statistikalar ===")
print(salary_category_stats)

# 4. Har bir State bo‘yicha
state_salary_stats = (
    population_df.groupby(["state", "SalaryCategory"])["salary"]
    .agg(
        PopulationCount="count",
        AverageSalary="mean",
        MedianSalary="median"
    )
    .reset_index()
)

state_totals = population_df.groupby("state")["salary"].count().reset_index(name="StateTotal")
state_salary_stats = state_salary_stats.merge(state_totals, on="state")
state_salary_stats["PopulationPercentage"] = (
    state_salary_stats["PopulationCount"] / state_salary_stats["StateTotal"] * 100
)

print("\n=== Har bir State bo'yicha Salary Band statistikalar ===")
print(state_salary_stats)


=== Umumiy Salary Band bo'yicha statistikalar ===
  SalaryCategory  PopulationCount  AverageSalary  MedianSalary  \
0           High            11616  995611.498020      993131.0   
1            Low                5     411.400000         377.0   
2         Medium               30    2848.133333        2533.0   

   PopulationPercentage  
0             99.699597  
1              0.042915  
2              0.257489  

=== Har bir State bo'yicha Salary Band statistikalar ===
            state SalaryCategory  PopulationCount  AverageSalary  \
0         Alabama           High              246   9.750741e+05   
1          Alaska           High               53   1.023254e+06   
2          Alaska         Medium                1   3.678000e+03   
3         Arizona           High              253   1.044695e+06   
4        Arkansas           High               60   1.130035e+06   
..            ...            ...              ...            ...   
73     Washington           High              2